In [8]:
import arcpy

# A list of features and coordinate pairs
feature_info = [[[1, 2], [2, 4], [3, 7]],
                [[6, 8], [5, 7], [7, 2], [9, 5]]]

# A list that will hold each of the Polyline objects
features = []

for feature in feature_info:
    # Create a Polyline object based on the array of points
    # Append to the list of Polyline objects
    features.append(
        arcpy.Polyline(
            arcpy.Array([arcpy.Point(*coords) for coords in feature])))

# Persist a copy of the Polyline objects using CopyFeatures
arcpy.CopyFeatures_management(features, r"C:\courses\spa\Week7\animaldata\data\polylinesTest.shp")

<Result 'C:\\courses\\spa\\Week7\\animaldata\\data\\polylinesTest.shp'>

In [28]:
#This one works
import arcpy, os

infc = r"C:\courses\spa\Week7\animaldata\data\GPS_points.shp"
infc2 = r"C:\courses\spa\Week7\animaldata\data\GPS_Tracks4.shp"

outputpathinfo = os.path.split(infc2)
print(outputpathinfo)

arcpy.env.overwriteOutput = True
#Create Empty feature Class to hold the line tracks
arcpy.CreateFeatureclass_management(outputpathinfo[0], outputpathinfo[1], "POLYLINE", infc, "#", "#", infc)

#read (open a search cursor) for the input points and loop through each point
cursor = arcpy.da.InsertCursor(infc2, ['SHAPE@', "Animal"])

panimal = -1
coords = []
for row in arcpy.da.SearchCursor(infc, ["SHAPE@XY", "Time", "Animal"], sql_clause=(None, "ORDER BY ANIMAL TIME DESC")):
    print(row[0], row[1], row[2])
    geom = row[0]

    if row[2] != panimal:
        print("create a new line for the next animal track AND write existing animal track to file")
        print(coords)
        if panimal != -1:
            outputGeom = arcpy.Polyline(arcpy.Array(coords))
            cursor.insertRow((outputGeom, panimal))
            coords = []
        
    coords.append(arcpy.Point(geom[0],geom[1]))
    print("add current point to animal (line track) we are currectly working on") 

    panimal = row[2]

#write (using a insert cursor) a line out into the new feature class everytime we encounter a new animal in the data.
outputGeom = arcpy.Polyline(arcpy.Array(coords))
cursor.insertRow((outputGeom, panimal))

#clean up (del cursors)

del cursor

('C:\\courses\\spa\\Week7\\animaldata\\data', 'GPS_Tracks4.shp')
(538615.7508954942, 1170796.9871083638) 1 1
create a new line for the next animal track AND write existing animal track to file
[]
add current point to animal (line track) we are currectly working on
(538495.1993566469, 1170796.9871083638) 2 1
add current point to animal (line track) we are currectly working on
(538495.1993566469, 1170622.1873770352) 3 1
add current point to animal (line track) we are currectly working on
(538615.7508954942, 1170619.173588564) 4 1
add current point to animal (line track) we are currectly working on
(538612.737107023, 1170697.5320888148) 5 1
add current point to animal (line track) we are currectly working on
(538549.4475491282, 1170703.559665757) 6 1
add current point to animal (line track) we are currectly working on
(538742.3300112841, 1170800.000896835) 7 3
create a new line for the next animal track AND write existing animal track to file
[<Point (538615.7508954942, 1170796.9871083638

In [27]:
#This one got updated wrong does not work

import arcpy

infc = r"C:\temp\spa\data\GPS_points.shp"
infc2 = r"C:\temp\spa\data\GPS_Tracks.shp"

def writeAnimal(pointList, animalnum):
    array = arcpy.Array(pointList)
    polyline = arcpy.Polyline(array)
    cursor.insertRow([polyline, animalnum]) 
    

arcpy.env.overwriteOutput = True
arcpy.CreateFeatureclass_management(r"C:\temp\spa\data", "GPS_Tracks.shp", "POLYLINE", infc, "#", "#", infc)

cursor = arcpy.da.InsertCursor(infc2, ['SHAPE@', "Animal"])

panimal = -1
pointList = []
# Enter for loop for each feature
for row in arcpy.da.SearchCursor(infc, ["SHAPE@XY", "Time", "Animal"], sql_clause=(None, "ORDER BY ANIMAL TIME DESC")):
    geom = row[0]
    animal = row[2]
    print(geom, row[1], row[2], panimal)
    if (animal != panimal) and (panimal != -1):
        print("Write out all points so far to another animal")
        print(pointList)
        writeAnimal(pointList, panimal)
        pointList = []
    pointList.append(arcpy.Point(geom[0],geom[1]))
    panimal = animal
    
writeAnimal(pointList, row[2])   

del cursor


<class 'arcgisscripting.ExecuteError'>: ERROR 000622: Failed to execute (Create Feature Class). Parameters are not valid.
ERROR 000628: Cannot set input into parameter spatial_reference.
